In [2]:
!pip install nltk gensim vaderSentiment numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.7 MB/s eta 0:00:00


In [12]:
# Install and import necessary libraries
!pip install scikit-learn tabulate

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate
from nltk.tokenize import word_tokenize

# **Preparation of the Dataset**

In [8]:
# Download dataset from T4SA
!wget --user=t4sa --password=U4Cm_dUa http://www.t4sa.it/dataset/t4sa_text_sentiment.tsv
!wget --user=t4sa --password=U4Cm_dUa http://www.t4sa.it/dataset/raw_tweets_text.csv

# Load datasets
sent_df = pd.read_csv('t4sa_text_sentiment.tsv', sep='\t')
sent_df['TWID'] = sent_df['TWID'].astype(str)

text_df = pd.read_csv('raw_tweets_text.csv')
text_df['id'] = text_df['id'].astype(str)

# Merge on IDs
merged_df = pd.merge(sent_df, text_df, left_on='TWID', right_on='id', how='inner')
merged_df = merged_df.drop(columns=['id'])

# Assign numeric label: NEG=0, NEU=1, POS=2
merged_df['label'] = merged_df[['NEG', 'NEU', 'POS']].idxmax(axis=1).map({
    'NEG': 0,
    'NEU': 1,
    'POS': 2
})

# Rename columns
merged_df = merged_df.rename(columns={'TWID': 'twitter_id', 'text': 'content'})

# Final dataset
final_df = merged_df[['twitter_id', 'label', 'content', 'NEG', 'NEU', 'POS']]

# Preview
print("Dataset size:", len(final_df))
print("\nFirst 20 rows (twitter_id, label, content):")
print(final_df[['twitter_id', 'label', 'content']].head(20))

print("\nSentiment label distribution:")
print(final_df['label'].value_counts().sort_index())

# Save merged dataset
final_df.to_csv('t4sa_merged_text_sentiment.csv', index=False)
print("\nMerged dataset saved as 't4sa_merged_text_sentiment.csv'")


--2025-12-14 07:15:12--  http://www.t4sa.it/dataset/t4sa_text_sentiment.tsv
Resolving www.t4sa.it (www.t4sa.it)... 146.48.85.151
Connecting to www.t4sa.it (www.t4sa.it)|146.48.85.151|:80... connected.
HTTP request sent, awaiting response... 401 Authorization Required
Authentication selected: Basic realm="T4SA Dataset"
Reusing existing connection to www.t4sa.it:80.
HTTP request sent, awaiting response... 200 OK
Length: 77271921 (74M) [text/tab-separated-values]
Saving to: ‘t4sa_text_sentiment.tsv’

t4sa_text_sentiment 100%[===================>]  73.69M  17.9MB/s    in 7.4s    

2025-12-14 07:15:20 (9.95 MB/s) - ‘t4sa_text_sentiment.tsv’ saved [77271921/77271921]

--2025-12-14 07:15:20--  http://www.t4sa.it/dataset/raw_tweets_text.csv
Resolving www.t4sa.it (www.t4sa.it)... 146.48.85.151
Connecting to www.t4sa.it (www.t4sa.it)|146.48.85.151|:80... connected.
HTTP request sent, awaiting response... 401 Authorization Required
Authentication selected: Basic realm="T4SA Dataset"
Reusing exist

# **Pipeline to evaluate lexicon based methods and tokenization methods**

In [13]:
# Load previously merged dataset
df = pd.read_csv('t4sa_merged_text_sentiment.csv')

N = 500
texts = df['content'][:N].tolist()
true_labels = df['label'][:N].tolist()


In [14]:
# ------------------------
# Generate predictions for all methods
# ------------------------
vader_preds, swn_preds, w2v_preds, ft_preds, glove_preds = [], [], [], [], []

for text in texts:
    tokens = word_tokenize(text.lower())
    # Lexicon-based
    vader_preds.append(vader_sentiment(text))
    swn_preds.append(sentiwordnet_sentiment(text))
    # Embedding-based
    w2v_preds.append(embedding_sentiment(tokens, word2vec_model.wv))
    ft_preds.append(embedding_sentiment(tokens, fasttext_model.wv))
    glove_preds.append(embedding_sentiment(tokens, glove_model))

In [15]:
# ------------------------
# Function to compute metrics
# ------------------------
def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    return acc, prec, rec, f1

In [16]:
# ------------------------
# Evaluation Table
# ------------------------
results = []
for text, true_label in zip(df['content'][:50], df['label'][:50]):  # Limit to first 50 for speed
    tokens = word_tokenize(text.lower())
    vader_label = vader_sentiment(text)
    swn_label = sentiwordnet_sentiment(text)
    w2v_label = embedding_sentiment(tokens, word2vec_model.wv)
    ft_label = embedding_sentiment(tokens, fasttext_model.wv)
    glove_label = embedding_sentiment(tokens, glove_model)

    results.append([text, true_label, vader_label, swn_label, w2v_label, ft_label, glove_label])



In [17]:
# Display as table
headers = ['Text', 'True', 'VADER', 'SentiWordNet', 'Word2Vec', 'FastText', 'GloVe']
print(tabulate(results, headers=headers, tablefmt='grid'))

+-----------------------------------------------------------------------------------------------------------------------------------------------------------+--------+---------+----------------+------------+------------+---------+
| Text                                                                                                                                                      |   True |   VADER |   SentiWordNet |   Word2Vec |   FastText |   GloVe |
+===========================================================================================================================================================+========+=========+================+============+============+=========+
| #Incredible #India #Atulya #Bharat - Land of Seekers #BeProud 🙏 🇮🇳  :|: Plz RT https://t.co/vpghReZWsa                                                   |      1 |       2 |              1 |          2 |          0 |       2 |
+--------------------------------------------------------------------------------

In [18]:
# ------------------------
# Compile evaluation
# ------------------------
methods = ['VADER', 'SentiWordNet', 'Word2Vec', 'FastText', 'GloVe']
predictions = [vader_preds, swn_preds, w2v_preds, ft_preds, glove_preds]
metrics_results = []

for method, preds in zip(methods, predictions):
    acc, prec, rec, f1 = compute_metrics(true_labels, preds)
    metrics_results.append([method, round(acc, 3), round(prec, 3), round(rec, 3), round(f1, 3)])

In [19]:
# Display summary table
headers = ['Method', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
print(tabulate(metrics_results, headers=headers, tablefmt='grid'))


+--------------+------------+-------------+----------+------------+
| Method       |   Accuracy |   Precision |   Recall |   F1-Score |
+==============+============+=============+==========+============+
| VADER        |      0.622 |       0.581 |    0.759 |      0.575 |
+--------------+------------+-------------+----------+------------+
| SentiWordNet |      0.736 |       0.54  |    0.559 |      0.547 |
+--------------+------------+-------------+----------+------------+
| Word2Vec     |      0.24  |       0.095 |    0.34  |      0.147 |
+--------------+------------+-------------+----------+------------+
| FastText     |      0.124 |       0.085 |    0.393 |      0.124 |
+--------------+------------+-------------+----------+------------+
| GloVe        |      0.244 |       0.304 |    0.326 |      0.138 |
+--------------+------------+-------------+----------+------------+
